In [1]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime

# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
my_region = boto3.session.Session().region_name # set the region of the instance

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + xgboost_container + " container for your SageMaker endpoint.")

Success - the MySageMakerInstance is in the eu-north-1 region. You will use the 669576153137.dkr.ecr.eu-north-1.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [3]:
bucket_name = 'ginopeppino' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [10]:
try:
  urllib.request.urlretrieve ("https://leonardo-michelazzo.github.io/LLama-AI/categorizzatore/Armani.csv", "prodotti.csv")
  print('Success: downloaded bank_clean.csv.')
except Exception as e:
  print('Data load error: ',e)

try:
  model_data = pd.read_csv('./prodotti.csv',index_col=0)
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [16]:
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(35, 13) (15, 13)


In [21]:
train_data.to_csv('train.csv',index=False,header=True)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [25]:
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(xgboost_container,role, instance_count=1, instance_type='ml.m5.4xlarge',output_path='s3://{}/{}/output'.format(bucket_name, prefix),sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,eta=0.2,gamma=4,min_child_weight=6,subsample=0.8,silent=0,objective='binary:logistic',num_round=100)

In [26]:
xgb.fit({'train': s3_input_train})

INFO:sagemaker:Creating training-job with name: xgboost-2023-06-22-14-16-53-169


2023-06-22 14:16:53 Starting - Starting the training job...
2023-06-22 14:17:09 Starting - Preparing the instances for training............
2023-06-22 14:18:59 Downloading - Downloading input data...
2023-06-22 14:19:30 Training - Training image download completed. Training in progress...Arguments: train
[2023-06-22:14:19:58:INFO] Running standalone xgboost training.
[2023-06-22:14:19:58:INFO] Path /opt/ml/input/data/validation does not exist!
[2023-06-22:14:19:58:INFO] File size need to be processed in the node: 0.03mb. Available memory size in the node: 54915.99mb
[2023-06-22:14:19:58:INFO] Determined delimiter of CSV input is ','
[14:19:58] S3DistributionType set as FullyReplicated
[2023-06-22:14:19:58:ERROR] Customer Error: Failed to load csv data with exception:
b"[14:19:58] src/data/csv_parser.h:124: Deduced delimiter ',' is not found in the line. Expected ',' as the delimiter to separate fields.\n\nStack trace returned 7 entries:\n[bt] (0) /usr/local/lib/python3.7/site-packages/

UnexpectedStatusException: Error for Training job xgboost-2023-06-22-14-16-53-169: Failed. Reason: ClientError: Failed to load csv data with exception:
b"[14:19:58] src/data/csv_parser.h:124: Deduced delimiter ',' is not found in the line. Expected ',' as the delimiter to separate fields.\n\nStack trace returned 7 entries:\n[bt] (0) /usr/local/lib/python3.7/site-packages/xgboost-0.71-py3.7.egg/xgboost/libxgboost.so(dmlc::StackTrace[abi:cxx11]()+0x3b) [0x7f6018326bbb]\n[bt] (1) /usr/local/lib/python3.7/site-packages/xgboost-0.71-py3.7.egg/xgboost/libxgboost.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x29) [0x7f60183272f9]\n[bt] (2) /usr/local/lib/python3.7/site-packages/xgboost-0.71-py3.7.egg/xgboost/libxgboost.so(dmlc::data::CSVParser<unsigned int, float>::ParseBlock(char const*, char const*, dmlc::data::RowBlockContainer<unsigned int, float>*)+0x3f9) [0x7f60184e4e99]\n[bt] (3) /usr/local/lib/python3.7/site-packages/xgboost-0.71-py3.7.egg/xgboost/libxgboost.so(+0x21d823) [0x7f60184ca